### Import dependency

In [1]:
import os
os.chdir('../')

import numpy as np
from tqdm import tqdm

# pip install natsort
import natsort # for index sorting

# pip install open3d
import open3d as o3d # for 3D Point cloud save & visualization

# should be located with same directory
from util.utility import npz_data, generate_pointcloud

%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
!pwd

/home/cvlab/Desktop/Event-based-3D-Reconstruction/depth2pc


### Load Depth map, Camara Parameter 

In [3]:
# PATH that contains *.npz files including camera parameters and depth map, etc.
path_dir = './data/circle/'
file_list = os.listdir(path_dir)
npz_list = []
for file_path in file_list:
    if os.path.splitext(file_path)[1] == '.npz':
        npz_list.append(file_path)

# Ascending sort
npz_list = natsort.natsorted(npz_list)
print('Ascending sorted by natsort.')
print(npz_list)


i_list=[] # List for virtual camera's intrinsic parameter
e_list=[] # List for virtual camera's extrinsic parameter with each frames
d_list=[] # List for virtual camera's depth map with each frames

### Extracting *.npz 
for npz in npz_list:
    i, e, d = npz_data(path_dir+npz)
    i_list.append(i)
    e[:,3]=(e[:,3]/1000)
    e_list.append(e)
    d_list.append(d)

Ascending sorted by natsort.
['0001.npz', '0002.npz', '0003.npz', '0004.npz', '0005.npz', '0006.npz', '0007.npz', '0008.npz', '0009.npz', '0010.npz', '0011.npz', '0012.npz', '0013.npz', '0014.npz', '0015.npz', '0016.npz', '0017.npz', '0018.npz', '0019.npz', '0020.npz', '0021.npz', '0022.npz', '0023.npz', '0024.npz', '0025.npz', '0026.npz', '0027.npz', '0028.npz', '0029.npz', '0030.npz', '0031.npz', '0032.npz', '0033.npz', '0034.npz', '0035.npz', '0036.npz', '0037.npz', '0038.npz', '0039.npz', '0040.npz', '0041.npz', '0042.npz', '0043.npz', '0044.npz', '0045.npz', '0046.npz', '0047.npz', '0048.npz', '0049.npz', '0050.npz', '0051.npz', '0052.npz', '0053.npz', '0054.npz', '0055.npz', '0056.npz', '0057.npz', '0058.npz', '0059.npz', '0060.npz', '0061.npz', '0062.npz', '0063.npz', '0064.npz', '0065.npz', '0066.npz', '0067.npz', '0068.npz', '0069.npz', '0070.npz', '0071.npz', '0072.npz', '0073.npz', '0074.npz', '0075.npz', '0076.npz', '0077.npz', '0078.npz', '0079.npz', '0080.npz', '0081.npz'

### Generate point cloud

In [4]:
result = []

print('Generate point cloud with %d input depth maps.' %(len(i_list)))
for i in tqdm(range(len(d_list))):
    # Read depth map, camera parameters
    # Generate point cloud with depth map, intrinsic parameters
    # Registrate point cloud by calculating of extrinsic parameters
    A = generate_pointcloud(d_list[i], i_list[i], e_list[i], 1000)

    if len(result) != 0: 
        result = np.vstack((A, result))
    else: 
        result = A

print('Point cloud generated with %d vertices.' %(len(result)))

Generate point cloud with 100 input depth maps.


100%|██████████| 100/100 [00:09<00:00, 10.62it/s]

Point cloud generated with 1167620 vertices.


### Extract point cloud with ply format

In [6]:
pcd_np = np.vstack(result)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_np)

# You can set your output_path and file name
output_path = "./output/"
file_name = "point_cloud.ply"
o3d.io.write_point_cloud(output_path + file_name, pcd)
print("%s successfully stored." %file_name)

point_cloud.ply successfully stored.
